In [1]:
import pandas as pd
from difflib import get_close_matches
import numpy as np
import requests
import json
import time
from config import cookies, headers

In [40]:
def close_matches(x, possible):
    matches = get_close_matches(x, possible, cutoff=0.80)
    if matches:
        return matches[0]
    else:
        return np.nan

def get_nfl_data(periodId):
    params = {
        "periodId": periodId,
        "site": "1",
        "sport": "1",
    }

    r = requests.get(
        "https://www.linestarapp.com/DesktopModules/DailyFantasyApi/API/Fantasy/GetSalariesV5",
        params=params,
        cookies=cookies,
        headers=headers,
    )
    r = r.json()
    # If there are no records, return None
    if len(r["Ownership"]["Salaries"]) == 0:
        raise ValueError(f"No data for periodId {periodId}")
    else:
        return r
    
def get_nfl_realized_slate(periodId):
    data = get_nfl_data(periodId)

    main_slate = [x for x in data["Ownership"]["Slates"] if x["SlateName"] == "Main"]
    # Raise errors if there are issues with selecting the right slate
    if len(main_slate) == 0:
        raise ValueError("No Main slate found")
    elif len(main_slate) > 1:
        raise ValueError("Multiple Main slates found")
    else:
        main_slate = main_slate[0]

    # Date of slate geames
    date = main_slate["SlateStart"][0:10]
    # Get SlateId for finding ownership data
    slate_id = [x["SlateId"] for x in main_slate["SlateGames"]][0]
    main_slate_game_ids = [x["GameId"] for x in main_slate["SlateGames"]]
    # Filter players to be those in games in the main slate, and have strictly positive projection
    slate_players = [
        x
        for x in data["Ownership"]["Salaries"]
        if (x["GID"] in main_slate_game_ids) & (x["PP"] > 0) & (x["POS"] != "K")
    ]

    # Make dictionary relating each player ID to projected ownership ammount
    player_ids = [x["PID"] for x in slate_players]
    proj_owned = {
        x["PlayerId"]: round(x["Owned"] / 100, 2)
        for x in data["Ownership"]["Projected"][str(slate_id)]
        if x["PlayerId"] in player_ids
    }
    # Get realized ownership for GPP tournaments that have contest type 4 on Linestar
    actual_owned = [
        x["OwnershipData"]
        for x in data["Ownership"]["ContestResults"]
        if (x["Contest"]["SlateId"] == slate_id) & (x["Contest"]["ContestType"] == 4)
    ][0]
    actual_owned = {
        x["PlayerId"]: round(x["Owned"] / 100, 2)
        for x in actual_owned
        if x["PlayerId"] in player_ids
    }

    for player in slate_players:
        try:
            # Adding projected ownership
            player["ProjOwned"] = proj_owned[player["PID"]]
        except KeyError:
            player["ProjOwned"] = 0.0

        try:
            # Adding realized ownership
            player["actual_owned"] = actual_owned[player["PID"]]
        except KeyError:
            player["actual_owned"] = 0.0

    # Make dictionaries with data we need
    slate_players = [
        {
            "Name": x["Name"],
            "Position": x["POS"],
            "Salary": x["SAL"],
            "Game": x["GI"],
            "Team": x["PTEAM"],
            "Opponent": x["OTEAM"],
            "Projection": x["PP"],
            "Scored": x["PS"],
            "pOwn": x["ProjOwned"],
            "actOwn": x["actual_owned"],
        }
        for x in slate_players
    ]

    frame = pd.DataFrame(slate_players)
    # Extract Game string
    frame["Game"] = frame["Game"].str.split(" ", expand=True)[0]
    return (date, frame)


def get_nfl_proj_slate(periodId):
    data = get_nfl_data(periodId)

    main_slate = [x for x in data["Ownership"]["Slates"] if x["SlateName"] == "Main"]
    # Raise errors if there are issues with selecting the right slate
    if len(main_slate) == 0:
        raise ValueError("No Main slate found")
    else:
        main_slate = main_slate[0]

    # Date of slate geames
    date = main_slate["SlateStart"][0:10]
    # Get SlateId for finding ownership data
    slate_id = [x["SlateId"] for x in main_slate["SlateGames"]][0]
    main_slate_game_ids = [x["GameId"] for x in main_slate["SlateGames"]]
    # Filter players to be those in games in the main slate, and have >0
    # projected points.
    slate_players = [
        x
        for x in data["Ownership"]["Salaries"]
        if (x["GID"] in main_slate_game_ids) & (x["PP"] > 0) & (x["POS"] != "K")
    ]
    # Construct dictionary relating player IDs to projected ownership
    player_ids = [x["PID"] for x in slate_players]
    proj_owned = {
        x["PlayerId"]: round(x["Owned"] / 100, 2)
        for x in data["Ownership"]["Projected"][str(slate_id)]
        if x["PlayerId"] in player_ids
    }

    for player in slate_players:
        try:
            # Adding projected ownership
            player["ProjOwned"] = proj_owned[player["PID"]]
        except KeyError:
            # If nothing found, assume 0
            player["ProjOwned"] = 0.0

    # Make dictionaries with data we need
    slate_players = [
        {
            "Name": x["Name"],
            "Position": x["POS"],
            "Salary": x["SAL"],
            "Game": x["GI"],
            "Team": x["PTEAM"],
            "Opponent": x["OTEAM"],
            "Projection": x["AggProj"],
            "pOwn": x["ProjOwned"],
        }
        for x in slate_players
    ]

    frame = pd.DataFrame(slate_players)
    # Extract Game string
    frame["Game"] = frame["Game"].str.split(" ", expand=True)[0]
    return (date, frame)

In [51]:
# Given a list of periodId's, write realized slates to file
periods = range(134, 300)

for period in periods:
    try:
        print(f"Getting period {period}")
        date, slate = get_nfl_realized_slate(period)
        print(f"Writing {date} data to file")
        slate.to_csv(f"./data/nfl_realized_slates/{date}.csv", index=False)
    except:
        print("Sleeping for 3 seconds")
        time.sleep(3)
        continue
    print("Sleeping for 3 seconds")
    time.sleep(3)

Getting period 134
Sleeping for 3 seconds
Getting period 135
Sleeping for 3 seconds
Getting period 136
Sleeping for 3 seconds
Getting period 137
Sleeping for 3 seconds
Getting period 138
Writing 2019-09-08 data to file
Sleeping for 3 seconds
Getting period 139
Writing 2019-09-15 data to file
Sleeping for 3 seconds
Getting period 140
Writing 2019-09-22 data to file
Sleeping for 3 seconds
Getting period 141
Writing 2019-09-29 data to file
Sleeping for 3 seconds
Getting period 142
Writing 2019-10-06 data to file
Sleeping for 3 seconds
Getting period 143
Writing 2019-10-13 data to file
Sleeping for 3 seconds
Getting period 144
Writing 2019-10-20 data to file
Sleeping for 3 seconds
Getting period 145
Writing 2019-10-27 data to file
Sleeping for 3 seconds
Getting period 146
Writing 2019-11-03 data to file
Sleeping for 3 seconds
Getting period 147
Writing 2019-11-10 data to file
Sleeping for 3 seconds
Getting period 148
Writing 2019-11-17 data to file
Sleeping for 3 seconds
Getting period 149

Writing 2021-12-12 data to file
Sleeping for 3 seconds
Getting period 290
Writing 2021-12-19 data to file
Sleeping for 3 seconds
Getting period 291
Writing 2021-12-26 data to file
Sleeping for 3 seconds
Getting period 292
Writing 2022-01-02 data to file
Sleeping for 3 seconds
Getting period 293
Writing 2022-01-09 data to file
Sleeping for 3 seconds
Getting period 294
Sleeping for 3 seconds
Getting period 295
Writing 2022-01-22 data to file
Sleeping for 3 seconds
Getting period 296
Writing 2022-01-30 data to file
Sleeping for 3 seconds
Getting period 297
Sleeping for 3 seconds
Getting period 298
Sleeping for 3 seconds
Getting period 299
Sleeping for 3 seconds


In [52]:
# Gathering all historical slates into one file
import os

frames = []
for file in os.listdir("./data/nfl_realized_slates"):
    data = pd.read_csv(f"./data/nfl_realized_slates/{file}")
    data["Date"] = file[0:10]
    frames.append(data)

In [60]:
test = pd.read_csv("./data/nfl_slates/2022-09-11.csv")